# Install Libraries

In [1]:
! pip install --upgrade datasets transformers peft evaluate accelerate
! pip show transformers


Name: transformers
Version: 4.57.3
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: C:\Users\arena\anaconda3\Lib\site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, sentence-transformers


In [2]:
from datasets import load_dataset
from datasets import DatasetDict
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from peft import LoraConfig, get_peft_model, TaskType
import evaluate
import numpy as np



# Data Preparation

In [3]:
raw = load_dataset("imdb")

train_valid = raw["train"].train_test_split(test_size=0.1, stratify_by_column="label", seed=42)
dataset = DatasetDict({
    "train": train_valid["train"],
    "validation": train_valid["test"],
    "test": raw["test"]
})


In [4]:
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_fn(batch):
    return tokenizer(
        batch["text"],
        padding=False,  
        truncation=True,
        max_length=128
    )

tokenized = dataset.map(tokenize_fn, batched=True, remove_columns=["text"])

small_train = tokenized["train"].shuffle(seed=42).select(range(2000))
small_valid = tokenized["validation"].shuffle(seed=42).select(range(500))
small_test = tokenized["test"].shuffle(seed=42).select(range(500))

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

# Base Model Selection and LoRA

In [5]:
num_labels = 2
base_model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint,
    num_labels=num_labels
)


lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q_lin", "v_lin"]
)

model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 739,586 || all params: 67,694,596 || trainable%: 1.0925


# Training

In [6]:
accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
recall = evaluate.load("recall")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "precision": precision.compute(predictions=preds, references=labels, average="binary")["precision"],
        "recall": recall.compute(predictions=preds, references=labels, average="binary")["recall"],
        "f1": f1.compute(predictions=preds, references=labels, average="binary")["f1"]
    }

collator = DataCollatorWithPadding(tokenizer)

args = TrainingArguments(
    output_dir="distilbert-imdb-lora",
    eval_strategy="epoch",  
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=50,
    learning_rate=2e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=1,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    fp16=False, 
    report_to="none"  
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=small_train,
    eval_dataset=small_valid,
    tokenizer=tokenizer,
    data_collator=collator,
    compute_metrics=compute_metrics
)

trainer.train()

C:\Users\arena\AppData\Local\Temp\ipykernel_8536\247035575.py:35: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
C:\Users\arena\anaconda3\envs\venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.484200,0.373253,0.838000,0.873469,0.810606,0.840864


TrainOutput(global_step=125, training_loss=0.5357499694824219, metrics={'train_runtime': 488.5607, 'train_samples_per_second': 4.094, 'train_steps_per_second': 0.256, 'total_flos': 67369703424000.0, 'train_loss': 0.5357499694824219, 'epoch': 1.0})

# Evaluation

In [7]:
test_metrics = trainer.evaluate(tokenized["test"])
print("Test metrics:", test_metrics)

C:\Users\arena\anaconda3\envs\venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Test metrics: {'eval_loss': 0.42685428261756897, 'eval_accuracy': 0.79948, 'eval_precision': 0.8246466048044402, 'eval_recall': 0.76072, 'eval_f1': 0.791394448836919, 'eval_runtime': 1340.4992, 'eval_samples_per_second': 18.65, 'eval_steps_per_second': 0.583, 'epoch': 1.0}


In [8]:
sentiment_pipeline = pipeline("text-classification", model=model, tokenizer=tokenizer)


text1 = "This movie was fantastic, I loved it!"
print(sentiment_pipeline(text1))


# Test case 2: Ambiguous
text2 = "The movie had great visuals but the story was boring."
print(sentiment_pipeline(text2))



Device set to use cpu


[{'label': 'LABEL_1', 'score': 0.9593409895896912}]
[{'label': 'LABEL_0', 'score': 0.7996536493301392}]
